# Mongo Agent

In [1]:
from smolagents.agents import ToolCallingAgent
from smolagents import tool, LiteLLMModel, CodeAgent
from pymongo import MongoClient
from datetime import datetime
from typing import List, Dict, Optional

C:\Works\AI\mongo-agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL_ID = "gpt-4o-mini"
MONGODB_URI = os.getenv("MONGO_URI")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
debug = os.getenv('DEBUG', 'False').lower() == 'true'

In [3]:
mongoclient = MongoClient(MONGODB_URI, appname="mongo-agent")
db = mongoclient.warehouse

In [4]:
@tool
def check_stock(product_id: str) -> Dict:
    """Query product stock level.

    Args:
        product_id: Product identifier

    Returns:
        Dict containing product details and quantity
    """
    return db.products.find_one({"_id": product_id})


@tool
def update_stock(product_id: str, quantity: int) -> bool:
    """Update product stock quantity.

    Args:
        product_id: Product identifier
        quantity: Amount to decrease from stock

    Returns:
        bool: Success status
    """
    result = db.products.update_one({"_id": product_id}, {"$inc": {"quantity": -quantity}})
    return result.modified_count > 0

In [5]:
@tool
def create_order(products: any, address: str) -> str:
    """Create new order for all provided products.

    Args:
        products: List of products with quantities
        address: Delivery address

    Returns:
        str: Order ID message
    """
    order = {"products": products, "status": "pending", "delivery_address": address, "created_at": datetime.now()}
    result = db.orders.insert_one(order)
    return f"Successfully ordered : {str(result.inserted_id)}"

In [6]:
from bson.objectid import ObjectId


@tool
def update_delivery_status(order_id: str, status: str) -> bool:
    """Update order delivery status to in_transit once a pending order is provided

    Args:
        order_id: Order identifier
        status: New delivery status is being set to in_transit or delivered

    Returns:
        bool: Success status
    """
    if status not in ["pending", "in_transit", "delivered", "cancelled"]:
        raise ValueError("Invalid delivery status")

    result = db.orders.update_one({"_id": ObjectId(order_id), "status": "pending"}, {"$set": {"status": status}})
    return result.modified_count > 0

In [17]:
class OrderManagementSystem:
    """Multi-agent order management system"""

    def __init__(self, model_id: str = MODEL_ID):
        self.model = LiteLLMModel(model_id=model_id, api_key=OPENAI_API_KEY)

        # Create managed agents
        self.inventory_agent = ToolCallingAgent(
            tools=[check_stock, update_stock], 
            model=self.model,
            name="inventory",
            description="Manages product inventory"
        )

        self.order_agent = ToolCallingAgent(
            tools=[create_order], 
            model=self.model,
            name="orders", 
            description="Handles order creation"
        )

        self.delivery_agent = ToolCallingAgent(
            tools=[update_delivery_status], 
            model=self.model,
            name="delivery",
            description="Manages delivery status"
        )

        # Put agents in a list for the manager
        self.managed_agents = [
            self.inventory_agent,
            self.order_agent, 
            self.delivery_agent
        ]

        # Create manager agent (remove system_prompt parameter)
        self.manager = CodeAgent(
            tools=[],
            model=self.model,
            managed_agents=self.managed_agents,
            additional_authorized_imports=["time", "json"],
        )

    def process_order(self, orders: List[Dict]) -> str:
        """Process a set of orders.

        Args:
            orders: List of orders each has address and products

        Returns:
            str: Processing result
        """
        # Include the instructions in the run() method instead
        instructions = """For each order:
        1. Create the order document
        2. Update the inventory
        3. Set delivery status to in_transit

        Use relevant agents to help you with these tasks.
        """
        
        return self.manager.run(
            f"{instructions}\n\n"
            f"Process the following {orders} as well as subtract the ordered items from inventory "
            f"to be delivered to relevant addresses"
        )

In [21]:
def add_sample_products():
    db.products.delete_many({})
    sample_products = [
        {"_id": "prod1", "name": "Laptop", "price": 999.99, "quantity": 10},
        {"_id": "prod2", "name": "Smartphone", "price": 599.99, "quantity": 15},
        {"_id": "prod3", "name": "Headphones", "price": 99.99, "quantity": 30},
    ]

    db.products.insert_many(sample_products)
    print("Sample products added successfully!")

In [22]:
add_sample_products()

Sample products added successfully!


In [23]:
system = OrderManagementSystem()

test_orders = [
    {
        "products": [{"product_id": "prod1", "quantity": 2}, {"product_id": "prod2", "quantity": 1}],
        "address": "123 Main St",
    },
    {"products": [{"product_id": "prod3", "quantity": 3}], "address": "456 Elm St"},
]

In [24]:
result = system.process_order(orders=test_orders)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ For each order:                                                                                                 │
│         1. Create the order document                                                                            │
│         2. Update the inventory                                                                                 │
│         3. Set delivery status to in_transit                                                                    │
│                                                                                                                 │
│         Use relevant agents to help you with these tasks.                                                       │
│                                                                                                                 │
│                                                                                                                 │
│ Process the following [{'products': [{'product_id': 'prod1', 'quantity': 2}, {'product_id': 'prod2',            │
│ 'quantity': 1}\], 'address': '123 Main St'}, {'products': [{'product_id': 'prod3', 'quantity': 3}\], 'address': │
│ '456 Elm St'}] as well as subtract the ordered items from inventory to be delivered to relevant addresses       │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  orders_data = [                                                                                                  
      {'products': [{'product_id': 'prod1', 'quantity': 2}, {'product_id': 'prod2', 'quantity': 1}], 'address':    
  '123 Main St'},                                                                                                  
      {'products': [{'product_id': 'prod3', 'quantity': 3}], 'address': '456 Elm St'}                              
  ]                                                                                                                
                                                                                                                   
  for order in orders_data:                                                                                        
      # Create the order document                                                                                  
      order_document = orders(f"Create order with {order['products']} to {order['address']}")                      
      print(f"Order document created: {order_document}")                                                           
                                                                                                                   
      # Update inventory                                                                                           
      for product in order['products']:                                                                            
          inventory_update = inventory(f"Subtract {product['quantity']} of {product['product_id']} from            
  inventory")                                                                                                      
          print(f"Inventory updated: {inventory_update}")                                                          
                                                                                                                   
      # Set delivery status to in_transit                                                                          
      delivery_status = delivery("Set delivery status to in_transit")                                              
      print(f"Delivery status updated: {delivery_status}")                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭─────────────────────────────────────────────── New run - orders ────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'orders'.                                                                          │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Create order with [{'product_id': 'prod1', 'quantity': 2}, {'product_id': 'prod2', 'quantity': 1}\] to 123 Main │
│ St                                                                                                              │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 2: Duration 15.42 seconds| Input tokens: 6,447 | Output tokens: 556]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_statement = "All orders have been successfully processed. Order IDs: 685b0167f592c87e9e37dff8 and          
  685b0185f592c87e9e37dff9 have been created, inventory updated, and delivery statuses set to 'in_transit'."       
  final_answer(final_statement)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: All orders have been successfully processed. Order IDs: 685b0167f592c87e9e37dff8 and 
685b0185f592c87e9e37dff9 have been created, inventory updated, and delivery statuses set to 'in_transit'.

[Step 3: Duration 5.15 seconds| Input tokens: 11,841 | Output tokens: 710]

In [25]:
print("Orders processing result:", result)

Orders processing result: All orders have been successfully processed. Order IDs: 685b0167f592c87e9e37dff8 and 685b0185f592c87e9e37dff9 have been created, inventory updated, and delivery statuses set to 'in_transit'.
